# Réseaux / Flots / Algorithme de Ford Fulkerson

## Réseaux

### Définition

On appelle *réseau* un graphe orienté, sans boucles ni arêtes multiples, dont les arêtes sont étiquetées par des nombres positifs. Par exemple, le graphe suivant est un réseau.

<img src="media/exemple_graph.png" />

On appelle *capacité d'une arête* l'étiquette qui lui est attachée. Par example, l'arête qui va de $5$ à $6$ a une capacité de $4$. On peut interpréter le réseau de cette façon : chaque arête est un *tuyau* reliant deux points, sa capacié est la taille du tuyau : le nombre d'éléments qui peuvent être transportés du premier point vers le second.

### Implantation

Un réseau s'implante d'une façon très similaire à un graphe. Les informations supplémentaires sont simplement *l'orientation des arêtes* et leur *capacité*. 

**Complétez la classe suivante qui représente un réseau. Vous êtes libres de stocker les données sous la forme que vous voulez cependant, on devra pouvoir créer le réseau soit avec une matrice, soit avec une liste d'éléments `(v1, v2, c)` où `v1` et `v2` sont des sommets et `c` la capacité de l'arête. Toutes les méthodes doivent correspondre à leur documentation.**

Remarque : pour un réseau, on considère qu'une arête de capacité 0 est la même chose qu'une arête qui n'existe pas.

In [60]:
class Network:

    def __init__(self, vertices = None, matrix = None, edges = None):
        """
        Initialisation du réseau

        INPUT :

            - vertices, un itérables sur les sommets du graphes
            - matrix, la matrice d'adjacence du réseau suivant les mêmes indices que `vertices`
            - edges, une liste de triplets (v1, v2, c) où v1 et v2 sont des sommets et c un nombre positif

        """
        if vertices is None:
            vertices = []
        
        # création d'un dictionnaire associant son indice à chaque sommet
        # (vous pouvez modifier si ça n'est pas utile à votre implantation)
        self._vertices = {vertices[i]: i for i in range(len(vertices))}

        # on ne peut pas donner à la fois matrix et edges
        if matrix is not None and edges is not None:
            raise ValueError("'matrix' et 'edges' ne peuvent pas être tous les deux initialisés")

        # initialisation différenciée : implantez les méthodes en question
        if matrix is not None:
            self._init_from_matrix(matrix)
        elif edges is not None:
            self._init_from_edges(edges)
        else:
            self._init_empty()

    def _init_from_matrix(self, matrix):
        """
        Initialisation à partir d'une matrice
        """
        # à compléter

    def _init_from_edges(self, edges):
        """
        Initialisation à partir d'un dictionnaire d'arêtes
        """
        # à compléter
        
    def _init_empty(self):
        """
        Initialisation d'un réseau vide (sans arêtes)
        """
        # à compléter
        
    def set_edge_capacity(self, v1, v2, c):
        """
        Donne la capacité `c` à l'arête `(v1,v2)`
        
        INPUT:
        
            - v1, un sommet du réseau
            - v2, un sommet du réseau
            - c la capacité de l'arête (v1,v2)
        """
        # à compléter
        
    def add_vertex(self, v):
        """
        Ajoute le sommet `v` au réseau
        """
        # à compléter
        
    def vertices(self):
        """
        Renvoie la liste des sommets du graphe
        """
        # à compléter
        
    def has_vertex(self, v1):
        """
        Renvoie vrai si v1 est un sommet du réseau
        
        INPUT:
        
            - v1, un sommet
        """
        # à compléter
        
    def edges(self):
        """
        Renvoie une liste de triplets `(v1,v2,c)` correspondant aux arêtes du réseau avec leur capacité. 
        """
        # à compléter
        
    def capacity(self, v1, v2):
        """
        Renvoie la capacité de l'arête (v1,v2) (si l'arête n'existe pas, la capacité est 0)
        
        INPUT:
        
            - v1, un sommet du réseau
            - v2, un sommet du réseau
        """
        # à compléter


In [61]:
# création d'un réseau vide
Network()

In [62]:
# création d'un réseau avec une unique arrête
Network(["a","b"], edges = [("a","b",3)])

In [63]:
Network(["a","b"], matrix = [[0,3],[0,0]])

In [64]:
# Ajout de sommets et d'arêtes
n = Network()
n.add_vertex("a")
n

In [65]:
n.add_vertex("b")
n

In [66]:
n.set_edge_capacity("a", "b", 3)
n

In [67]:
# Création du réseau donné en exemple
n1 = Network(range(7), edges = [(0,1,1), (0,2,2), (0,3,1), (1,4,3), (2,1,1), (2,0,1), (2,4,1), (3,5,2), (4,6,5), (5,2,1), (5,6,4), (6,3,3)])
n1

In [68]:
m = [
    [0,1,2,1,0,0,0],
    [0,0,0,0,3,0,0],
    [1,1,0,0,1,0,0],
    [0,0,0,0,0,2,0],
    [0,0,0,0,0,0,5],
    [0,0,1,0,0,0,4],
    [0,0,0,3,0,0,0]
]
n2 = Network(range(7), matrix = m)
n2

In [69]:
# quelques tests
assert n1 == n2
assert n1.capacity(3,5) == 2
assert n1.has_vertex(4)
assert not n1.has_vertex(7)
assert not n1.has_vertex("a")

In [70]:
# voisins
n1.neighbors_in(2) # 0 et 5

In [72]:
n1.neighbors_out(2) # 0, 1 et 4

In [73]:
assert set(n1.neighbors_in(3)) == {0,6}
assert n1.neighbors_out(3) == [5]

## Flots

### Définition

Un *flot* sur un réseau $r$ entre deux sommets $s$ et $t$ est une fonction $f$ qui associe un nombre à chaque arête de $r$ tel que:

 * $f(v1,v2)$ est toujours inférieur ou égal à capacité de de l'arête $(v1,v2)$ dans $r$;
 * pour tout sommet $a$ différent de $s$ et $t$, la somme des flots entrant en $a$ est égale à la somme des flots sortant de $a$.
 
Pour reprendre l'analogie précédente : si on imagine que le réseau est un ensemble de tuyaux de différentes tailles, le flot est une façon de faire "couler" des éléments entrant en $s$ vers le sommet $t$. En chaque point, le nombre d'éléments qui arrivent est égal au nombre d'éléments qui sort.

Voilà par exemple un flot sur le réseau donné en exemple précédemment (avec $s = 0$ et $t = 6$).

<img src="media/exemple_flot.png" />

Vous pouvez vérifier que le nombre écrit en rouge est toujours inférieur ou égal à la capacité de l'arête et que pour chaque point en dehors de 0 et 6, les flots entrants sont égaux aux flots sortant. Par exemple, sur le sommet $2$, on a en flot entrant $2+1$ et en flot sortant $1+1+1$.

Le sommet de départ est appelé la *source* et le sommet d'arrivée le *puit* ou la *cible*. Le flux net entrant sur la source est égal au flux net sortant de la cible, c'est ce qu'on appelle la *valeur* du flot. Dans l'exemple, on a un flot de valeur $0+2+1-1 = 2$ (flux entrant sur la source) qui est bien égal à $1+3-2 = 2$ (flux sortant de la cible).

### Implantation

En terme d'implantation, un flot est en fait un double réseau : le réseau de départ $r$ et un second réseau sur le même ensemble de sommets $f$ qui vérifie des propriétés particulières. On vous propose la classe suivante pertiellement implantée.

**Complétez les méthodes `flow_in`, `flow_out`, `check_capacity` et `check_in_out`**. (Les autres méthodes seront implantées plus tard).

**Vérifiez les exemples et les tests donnés**.

In [116]:
class Flow:
    
    def __init__(self, network, s, t, flow = None, check = True):
        """
        Initialisation à partir d'un réseau et d'un second réseau représentant le flot. Si le paramètre `flow` n'est pas initilisé, on crée un flot nul.
        
        INPUT:
        
            - network, un reseau Network
            - s, le sommet source
            - t, le sommet cible
            - flow (optionnel), un reseau Network
            - check (default = True), si True, un test de cohérence entre le flot et le réseau est lancé.
        """
        self._network = network
        self._s = s
        self._t = t
        if flow is None:
            self._flow = Network(network.vertices())
        else:
            self._flow = flow
            if check:
                assert flow.vertices() == network.vertices()
                assert self.check_capacity()
                assert self.check_in_out()
    
    def network(self):
        """
        Renvoie le réseau sur lequel on construit le flot
        """
        return self._network
    
    def source(self):
        """
        Renvoie la source du flot
        """
        return self._s
    
    def target(self):
        """
        Renvoie la cible du flot
        """
        return self._t
    
        
    def flows(self):
        """
        Renvoie une liste de triplets `(v1,v2,c)` correspondant aux arêtes du flot avec leur valeur. 
        """
        return self._flow.edges()
    

    def flow(self, v1, v2):
        """
        Renvoie le flot entre v1 et v2
        
        INPUT:
        
            - v1, un sommet du réseau
            - v2, un sommet du réseau
        """
        return self._flow.capacity(v1,v2)
    
    def set_flow(self, v1, v2, f):
        """
        Met la valeur du flot à `f` sur l'arête `(v1,v2)`
        
        INPUT :
        
            - v1, un sommet du réseau
            - v2, un sommet du réseau
            - f un nombre supérieur ou égal à 0
        """
        self._flow.set_edge_capacity(v1,v2,f)
        
    def add_flow(self, v1, v2, f):
        """
        Modifie la valeur du flot sur `(v1,v2)` de `f`
        
        INPUT :
        
            - v1, un sommet du réseau
            - v2, un sommet du réseau
            - f un nombre 
        """
        self._flow.set_edge_capacity(v1,v2,f + self.flow(v1,v2))
    
    def __repr__(self):
        """
        Renvoie une chaine de caractère représentant le flot
        """
        return "Flot sur le " + str(self._network) + ", flot : " + str(self._flow.edges())
    
    def __eq__(self, n2):
        """
        Renvoie vrai si n2 repésente le même flot que n1
        
        INPUT :
        
            - n2, un objet
        """
        if not isinstance(n2, Flow):
            return False
        return n2.network() == self.network() and n2._flow == self._flow
    
    def flow_in(self, v):
        """
        Renvoie la somme des flots entrant sur le sommet v
        """
        # à compléter
        
    def flow_out(self, v):
        """
        Renvoie la somme des flots sortant du sommet v
        """
        # à compléter
        
    def value(self):
        """
        Renvoie la valeur du flot (flux net entrant sur la source, ou flux net sortant de la cible)
        """
        # à compléter
    
    def check_capacity(self):
        """
        Renvoie vrai si la valeur du flot est bien inférieure ou égale à la capcité du réseau sur chacune des arêtes
        """
        # à compléter
        
    def check_in_out(self):
        """
        Renvoie vrai si la somme des flots entrant est égales à la somme des flots sortant sur chacun des sommets en dehors de la source et la cible
        """
        # à compléter
        
    ### Ford-Fulkerson ###
    
    def find_augmenting_path(self):
        """
        Cherche une chaine augmentante sur le flot :
        
            * si une chaine est trouvée, la méthode renvoie la chaine sous la forme d'une liste d'élément 
            `(v1,v2,p)` où `(v1,v2)` est l'arête de la chaine et `p` son potentiel avec le bon signe 
            (positif pour une arête dans le bon sens, négatif sinon)
            
            * si aucune chaîne n'est trouvée, la méthode renvoie None
        """
        # à compléter
        
    def increase_augmenting_path(self, path):
        """
        Modifie le flot en fonction de la chaine augmentante `path`
        
        INPUT:
        
            - path, une liste de triplet de la forme `(v1,v2,p)` où `(v1,v2)` est une arête du réseau et p est son potentiel d'augmentation (positif ou négatif)
            
        La méthode calcule la valeur minimale des valeurs absolues de p et modifie le flot en conséquence.
        """
        # à compléter


In [102]:
N = Network(range(7), edges = [(0,1,1), (0,2,2), (0,3,1), (1,4,3), (2,1,1), (2,0,1), (2,4,1), (3,5,2), (4,6,5), (5,2,1), (5,6,4), (6,3,3)])
F = Flow(N,0,6)
assert F.check_capacity()
assert F.check_in_out()
F

In [103]:
F.set_flow(0,2,1)
F

In [104]:
assert not F.check_in_out()

In [105]:
F.add_flow(0,2,3)
F

In [106]:
assert not F.check_capacity()

In [110]:
N = Network(range(7), edges = [(0,1,1), (0,2,2), (0,3,1), (1,4,3), (2,1,1), (2,0,1), (2,4,1), (3,5,2), (4,6,5), (5,2,1), (5,6,4), (6,3,3)])
N_f = Network(range(7), edges = [(0,1,1), (0,2,2), (1,4,2), (2,1,1), (2,0,1), (2,4,1), (3,5,2), (4,6,3), (5,2,1), (5,6,1), (6,3,2)])
F = Flow(N,0,6, N_f)
F

In [111]:
assert F.check_capacity()
assert F.check_in_out()
assert F.value() == 2

## Algorithme De Ford Fulkerson

L'algorithme de Ford Fulkerson sert à calculer *le flot de valeur maximale* (ou simplement *flot maximal*) sur un réseau entre deux points donnés. C'est-à-dire le nombre maximal d'éléments qu'on peut faire "couler" de la source vers la cible.

Le principe de base est simple : on part du flot nul (toutes les valeurs sont à 0) et on augmente le flot jusqu'à ce que ça ne soit plus possible. La question est de savoir comment "augmenter" un flot tout en conservant ses proriétés.

On propose ici une stratégie qu'on va implanter sur notre classe `Flow`.

### L'algorithme de la chaîne augmentante 

Une *chaîne augmentante* est un chemin non-orienté de $s$ à $t$ (on peut prendre les arêtes dans n'importe quel sens) tel que :

 * sur les arêtes prises dans "le bon sens", la valeur du flot est strictement inférieure à la capacité de l'arête
 * sur les arêtes prises dans "le mauvais sens", la valeur du flot est strictement supérieure à 0
 
Par exemple, $0 - 3 -6$ est une chaîne augmentante sur l'exemple de flot donné plus haut : l'arête $(0,3)$ prise dans le bon sens n'a pas atteint sa capacité maximale et l'arête $(3,6)$ prise dans le mauvais sens a un flot supérieur stricte à 0.

On regarde ensuite *le potentiel d'augmentation* $p$ pour chaque arête de la chaîne : 

 * pour une arête prise dans le bon sens, $p$ est égal à la différence entre la capacité de l'arête et la valeur du flot.
 * pour une arête prise dans le mauvais sens, $p$ est égal à la valeur du flot (la différence entre la valeur du flot et 0).
 
On détermine $pmin$ **le potentiel minimum sur la chaîne** puis, on **augmente** le flot des arêtes dans le bon sens et on **diminue** celui des arêtes dans le mauvais sens de la valeur $pmin$.

Par exemple, pour notre chaîne $0 - 3 - 6$, le potentiel de $(0,3)$ est 1 et le potentiel sur $(3,6)$ est 2. Dans ce cas, on a $pmin = 1$ et on peut : augmenter le flot de $(0,3)$ pour le faire passer à 1 et diminuer le flot de $(3,6)$ pour le faire passer à 1. Le résultat est toujours un flot.

**Implantez les méthodes `fing_augmenting_path` et `increase_augmenting_path` de la classe `Flow`.**

La méthode `find_augmenting_path` cherche une chaîne augmentante par un parcours en largeur du graphe en considérant que les arêtes qui vérifient la condition.

La méthode `increase_augmenting_path` prend une chaîne augmentante en paramètre et effectue la modification du flot en conséquence.



In [117]:
# vérification de find_augmenting_path
N = Network(range(7), edges = [(0,1,1), (0,2,2), (0,3,1), (1,4,3), (2,1,1), (2,0,1), (2,4,1), (3,5,2), (4,6,5), (5,2,1), (5,6,4), (6,3,3)])
N_f = Network(range(7), edges = [(0,1,1), (0,2,2), (1,4,2), (2,1,1), (2,0,1), (2,4,1), (3,5,2), (4,6,3), (5,2,1), (5,6,1), (6,3,2)])
F = Flow(N,0,6, N_f)
assert F.find_augmenting_path() == [(0, 3, 1), (6, 3, -2)]

In [124]:
# vérifiation de increase_augmenting_path
N = Network(range(7), edges = [(0,1,1), (0,2,2), (0,3,1), (1,4,3), (2,1,1), (2,0,1), (2,4,1), (3,5,2), (4,6,5), (5,2,1), (5,6,4), (6,3,3)])
N_f = Network(range(7), edges = [(0,1,1), (0,2,2), (1,4,2), (2,1,1), (2,0,1), (2,4,1), (3,5,2), (4,6,3), (5,2,1), (5,6,1), (6,3,2)])
F = Flow(N,0,6, N_f)
F.increase_augmenting_path([(0, 3, 1), (6, 3, -2)])
assert F.value() == 3
assert F.check_in_out()
assert F.check_capacity()

### Flot maximal

**Implantez la fonction suivante qui renvoie le flot maximal d'un réseau donné** : l'algortihme part du réseau null, cherche une chaîne augmentante, l'augmente puis recommence jusqu'à ce qu'il n'y ai plus de chaine augmentante.

In [142]:
def maximal_flow(network, s, t):
    """
    Renvoie le flot maximal sur le réseau `network` entre la source `s` et la cible `t`.
    
    INPUT :
    
        - network, un réseau de type Network
        - s, le sommet source
        - t, le sommet cible
    
    OUTPUT : un objet Flow de valeur maximale sur le réseau.
    """
    # écrire le code


In [127]:
N = Network(range(7), edges = [(0,1,1), (0,2,2), (0,3,1), (1,4,3), (2,1,1), (2,0,1), (2,4,1), (3,5,2), (4,6,5), (5,2,1), (5,6,4), (6,3,3)])
maximal_flow(N, 0, 6)

In [129]:
N = Network(range(7), edges = [(0,1,1), (0,2,2), (0,3,1), (1,4,3), (2,1,1), (2,0,1), (2,4,1), (3,5,2), (4,6,5), (5,2,1), (5,6,4), (6,3,3)])
F = maximal_flow(N,0,6)
assert F.value() == 4
assert F.check_capacity()
assert F.check_in_out()

In [144]:
N = Network(range(6), edges = [(0,1,10), (0,3,4), (1,2,13), (1,4,4), (2,5,10), (3,2,4), (4,5,4)])
F = maximal_flow(N,0,5)
assert F.value() == 14
assert F.check_capacity()
assert F.check_in_out()